# Python в инженерных расчетах. Лекция 09.
<hr style="border:2px solid gray">

#### План занятия:
1. **Понятие полного факторного эксперимента (ПФЭ)**
2. **Кодирование матрицы планирования ПФЭ**
3. **Расчет среднего значения и построчной дисперсии целевой функции**
4. **Проверка однородности дисперсий по критерию Кохрена**
5. **Расчет средней дисперсия воспроизводимости**
6. **Построение модели на основе метода наименьших квадратов**

### 1. Понятие полного факторного эксперимента (ПФЭ 2³)
<hr style="border:2px solid gray">

### Задача
Исследуем влияние трёх факторов на целевую переменную $ y $ — **предел текучести стали $ \sigma_{02} $, МПа**:
- $ x_1 $: содержание углерода (0.06% – 0.10%)
- $ x_2 $: толщина материала (0.7 мм – 1.2 мм)
- $ x_3 $: угол относительно направления проката (0° – 90°)

Построим модель в виде линейной регрессии:
$$
\hat{y} = b_0 + b_1 x_1 + b_2 x_2 + b_3 x_3
$$

<hr style="border:2px solid gray">

В ходе анализа проверим:
- Однородность дисперсий опытов — **критерий Кохрена**
- Значимость коэффициентов — **критерий Стьюдента**
- Адекватность модели — **критерий Фишера**

---

### Экспериментальные данные

Проведён **Полный Факторный Эксперимент (ПФЭ 2³)** — 3 фактора на 2 уровнях, всего 8 опытов.  
Каждый опыт повторён 3 раза для оценки ошибки воспроизводимости.

Данные получены для сталей **DC01** и **DC04**:

- **DC01**: $ C = 0.10\% $
- **DC04**: $ C = 0.06\% $

---

### Матрица планирования в натуральных переменных

| № опыта | Содержание углерода, % | Толщина, мм | Угол проката, ° | $ y_1 $, МПа | $ y_2 $, МПа | $ y_3 $, МПа |
|:-------:|:----------------------:|:-----------:|:---------------:|:------------:|:------------:|:------------:|
| 1       | 0.10                   | 0.7         | 0               | 238          | 237          | 239          |
| 2       | 0.06                   | 0.7         | 0               | 160          | 161          | 159          |
| 3       | 0.10                   | 1.2         | 0               | 240          | 241          | 239          |
| 4       | 0.06                   | 1.2         | 0               | 186          | 185          | 187          |
| 5       | 0.10                   | 0.7         | 90              | 243          | 244          | 242          |
| 6       | 0.06                   | 0.7         | 90              | 164          | 163          | 165          |
| 7       | 0.10                   | 1.2         | 90              | 242          | 243          | 241          |
| 8       | 0.06                   | 1.2         | 90              | 201          | 200          | 202          |


In [1]:
import numpy as np
from Lection_009 import CochranTable, StudentTable, FisherTable, Full_factorial_matrix, Regression

# Настройки отображения
np.set_printoptions(precision=4, suppress=True)

## 2.  Матрица планирования кодируется таким образом, что:
- низкий уровень → -1 (например, если содержание углерода равно 0.06%) 
- высокий уровень → +1 (например, если содержание углерода равно 0.1%)

 **Общий вид кодирования фактора:**
$$
x_j = \frac{X_j - X_{j0}}{\Delta_j}
$$
где:
- $ x_j $ — кодированное значение фактора (безразмерное, $ -1 $ или $ +1 $)
- $ X_j $ — натуральное значение фактора
- $ X_{j0} = \frac{X_{j\text{max}} + X_{j\text{min}}}{2} $ — **центр уровня** (середина интервала)
- $ \Delta_j = \frac{X_{j\text{max}} - X_{j\text{min}}}{2} $ — **интервал варьирования**

### Для нашего случая

### Для содержания углерода (C):
- $X_{\text{min}} = 0.06\%$, $X_{\text{max}} = 0.10\%$
- **Центр уровня**: $X_{10} = \frac{0.10 + 0.06}{2} = 0.08\%$
- **Интервал варьирования**: $\Delta_1 = \frac{0.10 - 0.06}{2} = 0.02\%$

### Для толщины (h):
- $X_{\text{min}} = 0.7$ мм, $X_{\text{max}} = 1.2$ мм  
- **Центр уровня**: $X_{20} = \frac{1.2 + 0.7}{2} = 0.95$ мм
- **Интервал варьирования**: $\Delta_2 = \frac{1.2 - 0.7}{2} = 0.25$ мм

### Для угла проката (θ):
- $X_{\text{min}} = 0°$, $X_{\text{max}} = 90°$
- **Центр уровня**: $X_{30} = \frac{90 + 0}{2} = 45°$
- **Интервал варьирования**: $\Delta_3 = \frac{90 - 0}{2} = 45°$

## Формулы кодирования:

$$
x_1 = \frac{C - 0.08}{0.02},\quad
x_2 = \frac{h - 0.95}{0.25},\quad
x_3 = \frac{\theta - 45}{45}
$$

In [2]:
# Порядок: x1 (C), x2 (h), x3 (θ)
# +1: верхний уровень, -1: нижний уровень
matrix = Full_factorial_matrix(3)
print("Матрица планирования в кодированном масштабе:")
print("x1 (C)  x2 (h)  x3 (θ)")
print(matrix.matrix_normalized)

Матрица планирования в кодированном масштабе:
x1 (C)  x2 (h)  x3 (θ)
[[ 1 -1 -1]
 [-1 -1 -1]
 [ 1  1 -1]
 [-1  1 -1]
 [ 1 -1  1]
 [-1 -1  1]
 [ 1  1  1]
 [-1  1  1]]


In [3]:
# Значения целевой переменной: каждая строка — 3 повторения одного и того же из восьми опытов
y_repeated = np.array([
    [238.5, 237.0, 239.2],  # Опыт 1: C=0.10%, h=0.7 мм, θ=0°  → DC01
    [160.9, 161.4, 158.0],  # Опыт 2: C=0.06%, h=0.7 мм, θ=0°  → DC04
    [240.2, 241.9, 239.0],  # Опыт 3: C=0.10%, h=1.2 мм, θ=0° → DC01
    [183.1, 185.4, 187.8],  # Опыт 4: C=0.06%, h=1.2 мм, θ=0° → DC04
    [241.4, 244.8, 242.3],  # Опыт 5: C=0.10%, h=0.7 мм, θ=90° → DC01
    [165.6, 163.2, 165.5],  # Опыт 6: C=0.06%, h=0.7 мм, θ=90° → DC04
    [243.1, 243.1, 241.2],  # Опыт 7: C=0.10%, h=1.2 мм, θ=90° → DC01
    [201.9, 198.3, 202.1],  # Опыт 8: C=0.06%, h=1.2 мм, θ=90° → DC04
])

# для дальнейшей регрессии создаем обхект regression и в качестве атрибутов подаем в него матрицу в кодированном масштабе и целевую переменную
regression = Regression(matrix.matrix_normalized, y_repeated)

print("Отклики с повторениями:")
print(regression.target_values)

Отклики с повторениями:
[[238.5 237.  239.2]
 [160.9 161.4 158. ]
 [240.2 241.9 239. ]
 [183.1 185.4 187.8]
 [241.4 244.8 242.3]
 [165.6 163.2 165.5]
 [243.1 243.1 241.2]
 [201.9 198.3 202.1]]


## 3.  Расчет среднего значения и построчной дисперсии

После проведения эксперимента, где **каждый опыт повторяется несколько раз** (в нашем случае — 3 раза), важно не просто записать результаты, а **охарактеризовать каждый опыт двумя показателями**:

1. **Среднее значение отклика** — чтобы понять, *что показал опыт в среднем*.
2. **Построчная дисперсия** — чтобы понять, *насколько стабильны измерения*.

### Среднее значение целевой переменной

**Формула:**
$$
\bar{y}_i = \frac{1}{m} \sum_{j=1}^m y_{ij}
$$

Где:
- $ \bar{y}_i $ — среднее значение отклика для **i-го опыта**
- $ m $ — число повторений (в нашем случае $ m = 3 $)
- $ y_{ij} $ — j-е измерение в i-м опыте



### Построчная дисперсия

**Формула:**
$$
S_i^2 = \frac{1}{m-1} \sum_{j=1}^m (y_{ij} - \bar{y}_i)^2
$$

Где:
- $ S_i^2 $ — дисперсия i-го опыта
- $ m-1 $ — число степеней свободы (для несмещённой оценки)
- $ y_{ij} - \bar{y}_i $ — отклонение каждого измерения от среднего


Почему именно $ m-1 $?

При расчёте построчной дисперсии мы используем **среднее, вычисленное по тем же данным**, поэтому теряется одна степень свободы.  
Деление на $ m-1 $ (а не на $ m $) даёт **несмещённую оценку** истинной дисперсии.


**Зачем нужна построчная дисперсия:**
- Показывает **разброс данных** в рамках одного опыта.
- Чем больше дисперсия — тем **менее стабилен процесс** при этих факторах.
- Используется для:
  - Проверки **однородности дисперсий** (критерий Кохрена)
  - Оценки **ошибки эксперимента**
  - Проверки **адекватности модели** (критерий Фишера)

>⚠️ Если дисперсия одного опыта сильно больше других — возможно, в нём была ошибка (например, сбой оборудования).

In [4]:
# число опытов и число посторений расчитывается автоматически по размерам матрицы планирования и матрицы отклика
print (f'Количество опытов: {regression.n_samples}')
print (f'Количество повторений: {regression.n_repeats}')


Количество опытов: 8
Количество повторений: 3


In [5]:
# Расчет построчной дисперсии целевой переменной и дисперсии воспроизводимости
regression.get_S2()
print ('Среднее значение и построчная дисперсия')
for i in range (len(regression.S2)):
    print (f'Опыт № {i}: {regression.target_values_avg[i]:.3f}   {regression.S2[i]:.3f}')

✅ Дисперсии однородны. Можно продолжать анализ.
Среднее значение и построчная дисперсия
Опыт № 0: 238.233   1.263
Опыт № 1: 160.100   3.370
Опыт № 2: 240.367   2.123
Опыт № 3: 185.433   5.523
Опыт № 4: 242.833   3.103
Опыт № 5: 164.767   1.843
Опыт № 6: 242.467   1.203
Опыт № 7: 200.767   4.573


## 4. Проверка однородности дисперсий по критерию Кохрена

$$
G = \frac{\max(S_i^2)}{\sum_{i=1}^n S_i^2}
$$

Полученное расчетное значение критерия Кохрена необходимо сравнить с табличным при $ \alpha = 0.05 $:
| $ n $ ↓ \\ $ f $ → | 2       | 3       | 4       | 5       | 6       |
|---------------------|---------|---------|---------|---------|---------|
| **2**               | 0.998   | 0.967   | 0.906   | 0.841   | 0.781   |
| **3**               | 0.975   | 0.874   | 0.768   | 0.684   | 0.623   |
| **4**               | 0.939   | 0.798   | 0.684   | 0.600   | 0.539   |
| **5**               | 0.906   | 0.746   | 0.633   | 0.551   | 0.494   |
| **6**               | 0.877   | 0.707   | 0.599   | 0.520   | 0.465   |
| **7**               | 0.850   | 0.677   | 0.574   | 0.498   | 0.444   |
| **8**               | 0.828   | 0.653   | 0.554   | 0.479   | 0.427   |
| **9**               | 0.808   | 0.633   | 0.537   | 0.463   | 0.412   |
| **10**              | 0.791   | 0.615   | 0.522   | 0.450   | 0.401   |

- $ n $ — число опытов (или число групп дисперсий)
- $ f = m - 1 $ — число степеней свободы **одной** выборки (где $ m $ — число повторений)
- $ \alpha $ - уровень значимости

$ \alpha = 0.05 $ означает, что мы допускаем 5% вероятность ошибочно заключить, что дисперсии неоднородны.

> 🔹 Пример использования приведенной выше таблицы:  
> У нас 8 опытов ($ n = 8 $), каждый повторён 3 раза → $ f = m - 1 = 2 $  
> Тогда критическое значение: $ G_{\text{крит}} = 0.828 $  
> Если расчетное $ G_{\text{набл}} < 0.828 $ → дисперсии однородны.

In [6]:
# Расчетный критерий Кохрена расчитывается автоматически при вызове метода get_S2
print (f'Расчетный критерий Кохрена: {regression.cochran}')
print (f'Табличный критерий Кохрени: {regression.cochran_max}')
if regression.cochran < regression.cochran_max:
    print ('Все ОК, дисперсия однородна.')
else:
    print ('Что-то пошло не так, проверка однородности дисперсии по критерию Кохрена не пройдена.')


Расчетный критерий Кохрена: 0.24011012896681763
Табличный критерий Кохрени: 0.5157
Все ОК, дисперсия однородна.


## 5. Расчет средней дисперсия воспроизводимости (сумма построчных дисперсий)

$$
S^2_{\text{воспр}} = \frac{1}{n} \sum_{i=1}^n S_i^2
$$

понадобится нам чуть позднее

In [7]:
# Расчитывается в методе get_S2
print(f"Средняя дисперсия воспроизводимости: {regression.S2_vospr:.3f}")

Средняя дисперсия воспроизводимости: 2.875


## 6. Построение модели на основе метода наименьших квадратов

В матрица плана X необходимо добавить столбец единиц для последующего рассчета свободного члена уравнения регрессии (коэффициента $ b_0$):
$$
\mathbf{X} = [\mathbf{1},\ \mathbf{x}_1,\ \mathbf{x}_2,\ \mathbf{x}_3]
$$

Коэффициенты линейной регрессии расчитываются с помощью следующего матричного уравнения:
$$
\mathbf{b} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}
$$

In [8]:
# добавляем столбец с единицами в матрицу планирования в кодированном масштабе
_ = regression.get_coefficients()

print("Коэффициенты модели:")
for i in range (len(regression.coefs)):
    print (f'b{i}: {regression.coefs[i]:.2f}')

Коэффициенты модели:
b0: 209.37
b1: 31.60
b2: 7.89
b3: 3.34
